In [10]:
import pandas as pd
import gradio as gr
import nltk
import numpy as np
from search import search_with_query

In [11]:
df_descripteur = pd.read_csv('freq_poids_porter.csv')

In [12]:
df_inverse = pd.read_csv('df_poids.csv').drop('Unnamed: 0', axis=1)

In [13]:
def search_document_descripteur(document):
        new_df = df_descripteur[df_descripteur['Document']== document]
        new_df.drop(['Document'], axis=1, inplace=True)
        
        inverse = df_inverse.iloc[int(document)].to_frame()
        inverse = inverse[(inverse.T != 0.000000).all()]
        inverse = inverse.reset_index(level=0)
        inverse["Terme"] = inverse["index"]
        inverse["Poid"] = inverse[int(document)]
        inverse.drop([int(document)], axis=1, inplace=True)
        inverse.drop(['index'], axis=1, inplace=True)

        return [new_df, inverse]

In [14]:
def search_document_inverse(phrase):
        ExpReg = nltk. RegexpTokenizer('(?:[A-Za-z]\.)+|\d+(?:\.\d+)?%?|\w+(?:\-\w+)*')
        termes = ExpReg.tokenize(phrase)
        termes = [t.lower() for t in termes]
        MotsVides = nltk.corpus.stopwords.words('english')
        Porter = nltk.PorterStemmer()
        TermesSansMotsVides_p = [Porter.stem(terme) for terme in termes if terme.lower() not in MotsVides]
        
        new_df = df_inverse[TermesSansMotsVides_p]
        new_df = new_df[(new_df.T != 0.000000).any()]
        new_df = new_df.reset_index(level=0)

        desc_df = df_descripteur.loc[df_descripteur['Terme'].isin(TermesSansMotsVides_p) ]

        #new_df.drop(['document'], axis=1, inplace=True)
        return new_df, desc_df

In [15]:
def search_query(query):
        results = search_with_query(query)
        return pd.DataFrame(results)
        

In [16]:
# search_query("information AND classification OR NOT title AND computers")

In [17]:
with gr.Blocks() as app:
        gr.Markdown("main app")
        with gr.Tab("Main Searches"):
                with gr.Row():
                        with gr.Column():
                                gr.Markdown("Search by Term")
                                search_input = gr.Textbox(label="Search for terms")
                                search_button = gr.Button(label="Search by Term")
                with gr.Row():
                        with gr.Column():
                                gr.Markdown("Search by Document")
                                search_document = gr.Number()
                                search_document_button = gr.Button(label="Search by Document")
                with gr.Row():
                        with gr.Column():
                                gr.Markdown("Descripteur")
                                search_output_descripteur = gr.Numpy()
                        with gr.Column():
                                gr.Markdown("Inverse")
                                search_output_inverse = gr.Numpy()
        with gr.Tab("Boolean Search"):
                gr.Markdown("Boolean Search")
                search_boolean = gr.Textbox(label="Boolean search")
                search_boolean_button = gr.Button(label="Search")
                search_boolean_output = gr.Numpy()
        with gr.Tab("DBScan"):
                gr.Markdown("DBScan")
        with gr.Tab("Naive Bayes"):
                gr.Markdown("Naive Bayes")
        search_button.click(search_document_inverse, inputs = search_input, outputs = [search_output_inverse, search_output_descripteur])
        search_document_button.click(search_document_descripteur, inputs = search_document, outputs = [search_output_descripteur, search_output_inverse])
        search_boolean_button.click(search_query, inputs = search_boolean, outputs = search_boolean_output)

app.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
